## Assignment 1: Word Count

In this assignment, you will identify the most popular words appearing in an academic paper written by me and Joson Michaud **Detecting Covid19 Fake News**. You will create a word cloud to visualize the results. You will first upload the file covid19_fake_news_detection.txt to the hpc server. Please complete the five questions at the end of this notebook.

In [ ]:
from pyspark.sql import SparkSession

# change the account name to your email account
account='sli'

# define a data path to access the data in isa460 folder
data_path='/net/clusterhn/home/'+account+'/isa460/data/'

spark = SparkSession.builder.appName("Word Count")\
        .config("spark.port.maxRetries", "100")\
        .getOrCreate()

# confiture the log level (defaulty is WWARN)
spark.sparkContext.setLogLevel('ERROR')

# read the csv file

book = spark.read.text(data_path+"covid19_fake_news_detection.txt")

In [ ]:
book.show(10, truncate=50)

### Below code shows the top 10 words based on frequency after changing all words to lowercase, remove null value and special characters)

In [ ]:
from pyspark.sql.functions import (
    col,
    explode,
    lower,
    regexp_extract,
    split,
)

lines = book.select(split(book.value, " ").alias("line"))
 
words = lines.select(explode(col("line")).alias("word"))
 
words_lower = words.select(lower(col("word")).alias("word"))
 
words_clean = words_lower.select(
    regexp_extract(col("word"), "[a-z']*", 0).alias("word")
)
 
words_nonull = words_clean.where(col("word") != "")
 
results = words_nonull.groupby(col("word")).count()
 
results.orderBy("count", ascending=False).show(10)

## 1. Modify the orignal code to return the number of the distinct words in this article

## 2. Modify the origianl code to return a smaple of 10 words that only appreas once

## 3. Modify original code to remove all words having less than 3 letters, show top 20 words.

## 4. Modify the above code to remove the stopwords (the, and, for, this, that, was, are) after removing words having less than 3 letters,  save the top 100 words in a dataframe

## 5. Create a word cloud to visualize the top 100 words (hint: you may need to install other package, to install a package within the notebook, use 
!pip install packagename